In [1]:
from datetime import datetime
import os
import gzip
import pandas as pd
import numpy as np

today = datetime.today().strftime('%Y-%m-%d')

os.chdir(r'D:\TD\FMF\Strategies')
filename = 'StrategyTestingResults v2.csv.gz'

In [2]:
###Load the data

with gzip.open(filename, 'rt') as file:
    data = pd.read_csv(file, parse_dates=['entry_time', 'exit_time'], index_col=None, dayfirst=True)
    data['entry_time'] = pd.to_datetime(data['entry_time'])#, dayfirst=True)
    data['exit_time'] = pd.to_datetime(data['exit_time'])#, dayfirst=True)
    data = data.sort_values(['entry_time', 'exit_time'], ascending = (True, True))
    data.dropna(axis=0, how='all', inplace=True)
    data = data.drop_duplicates(subset=['ticker', 'timeframe', 'strategy', 'entry_price', 'entry_time', 
                'exit_price', 'exit_time'], keep='last', inplace=False)#, ignore_index=True)
    #data['profit%'] = np.where(data['entry_signal'] == 'BUY', (((((10000 * 0.992) / data['entry_price']) * data['exit_price']) - (10000 * 0.996)) / 100), (((10000 * 0.996) - (((10000 * 0.992) / data['entry_price']) * data['exit_price'])) / 100)) #I am using 0.992 since the second trasaction uses the amount after the first commission and a second .004 commission is added.
    data['profit%'] = np.where(data['entry_signal'] == 'BUY', ((data['exit_price'] - data['entry_price']) * 10000 - 8) / 10000, ((data['entry_price'] - data['exit_price']) * 10000 - 8) / 10000) #made it simpler here: Binance is charging me 8 USD for each 10k transaction

#Computing the cummulative profit (includes the trade direction: BUY or SELL). This basically adds up the profit after each trade.
data['CumulativeProfit%'] = data.groupby(['ticker', 'strategy', 'timeframe', 'entry_signal'])['profit%'].cumsum() / 100
data['CumulativeProfit%'] = data['CumulativeProfit%'].apply(lambda x: f'{x:.2%}')

#Computing the cumulative profit by strategy without the direction of the trade.
data['CumulativeProfitStrategy%'] = data.groupby(['ticker', 'strategy', 'timeframe'])['profit%'].cumsum() / 100
data['CumulativeProfitStrategy%'] = data['CumulativeProfitStrategy%'].apply(lambda x: f'{x:.2%}')

#Computing how long the trade was in the market
data['TimeInMarket'] = (data['exit_time'] - data['entry_time']).dt.total_seconds() / 3600.0

#Getting the day of the week when the trade was placed
data['WeekDayEntry'] = data['entry_time'].dt.dayofweek

#Getting the hour when the trade was placed
data['HourOfEntry'] = data['entry_time'].dt.hour

#Checking the data
data

,ticker,timeframe,strategy,entry_price,entry_time,entry_signal,exit_price,exit_time,exit_signal,runup%,drawdown%,profit%,CumulativeProfit%,CumulativeProfitStrategy%,TimeInMarket,WeekDayEntry,HourOfEntry
0,BINANCE:BLUEBIRDUSDT.P,1s,Strategy v2.1,6.777,2023-07-04 01:19:00,BUY,6.807,2023-07-04 02:43:00,CLOSE BUY,0.44,0.48,0.0292,0.03%,0.03%,1.400000,1,1
1,BINANCE:BLUEBIRDUSDT.P,1s,Strategy v2.1,6.783,2023-07-04 01:19:00,BUY,6.807,2023-07-04 02:43:00,CLOSE BUY,0.35,0.56,0.0232,0.05%,0.05%,1.400000,1,1
2,BINANCE:BLUEBIRDUSDT.P,1s,Strategy v2.1,6.786,2023-07-04 01:19:00,BUY,6.807,2023-07-04 02:43:00,CLOSE BUY,0.31,0.61,0.0202,0.07%,0.07%,1.400000,1,1
3,BINANCE:BLUEBIRDUSDT.P,1s,Strategy v2.1,6.788,2023-07-04 01:25:00,BUY,6.807,2023-07-04 02:43:00,CLOSE BUY,0.28,0.64,0.0182,0.09%,0.09%,1.300000,1,1
4,BINANCE:BLUEBIRDUSDT.P,1s,Strategy v2.1,6.787,2023-07-04 01:26:00,BUY,6.807,2023-07-04 02:43:00,CLOSE BUY,0.29,0.62,0.0192,0.11%,0.11%,1.283333,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325844,BINANCE:BNBUSDT.P,1m,[CP] ChannelBreakOutStrategy,205.470,2023-10-12 14:52:00,SELL,205.570,2023-10-12 14:56:00,BUY,0.07,0.05,-0.1008,144.27%,268.95%,0.066667,3,14
325845,BINANCE:BNBUSDT.P,1m,[CP] ChannelBreakOutStrategy,205.570,2023-10-12 14:56:00,BUY,205.500,2023-10-12 15:02:00,SELL,0.04,0.04,-0.0708,124.61%,268.88%,0.100000,3,14
325846,BINANCE:BNBUSDT.P,1m,[CP] Momentum Strategy,205.590,2023-10-12 14:57:00,BUY,205.590,2023-10-12 15:00:00,SELL,0.03,0.02,-0.0008,77.03%,169.74%,0.050000,3,14
325847,BINANCE:BNBUSDT.P,1m,[CP] Momentum Strategy,205.590,2023-10-12 15:00:00,SELL,205.550,2023-10-12 15:05:00,BUY,0.13,0.00,0.0392,92.76%,169.78%,0.083333,3,15


In [3]:
#Cleaning the data so that the COMMENTS ARE ALWAYS BUY/SELL, CLOSE BUY/CLOSE SELL

data['entry_signal'] = data['entry_signal'].apply(lambda x: x.replace('Entry Long', 'BUY'))
data['entry_signal'] = data['entry_signal'].apply(lambda x: x.replace('Entry Short', 'SELL'))
data['exit_signal'] = data['exit_signal'].apply(lambda x: x.replace('Exit Long', 'CLOSE BUY'))
data['exit_signal'] = data['exit_signal'].apply(lambda x: x.replace('Exit Short', 'CLOSE SELL'))
data['entry_signal'] = data['entry_signal'].apply(lambda x: x.replace('MomLE', 'BUY'))
data['entry_signal'] = data['entry_signal'].apply(lambda x: x.replace('MomSE', 'SELL'))
data['exit_signal'] = data['exit_signal'].apply(lambda x: x.replace('MomLE', 'SELL'))
data['exit_signal'] = data['exit_signal'].apply(lambda x: x.replace('MomSE', 'BUY'))


In [4]:
#####RESAVE DATA TO CSV
os.chdir(r'D:\TD\FMF\Strategies')
file = 'StrategyTestingResults v2 for Analysis.csv.gz'
data.to_csv(file, index=False, compression='gzip')

#############################################<b>OverallStrategy Processing</b>#####################################

In [5]:
######Summing up the profit for each tuple 'ticker, strategy and timeframe' in order to compare them. 
totalprofit2 = pd.DataFrame()
totalprofit2 = data.groupby(['ticker', 'strategy', 'timeframe'])['profit%'].sum().reset_index()
totalprofit2.columns = ['ticker', 'strategy', 'timeframe', 'TotalProfit%']

#Getting the average profit per trade for each tuple of ticker, strategy, timeframe
avgprofit2 = data.groupby(['ticker', 'strategy', 'timeframe'])['profit%'].mean().reset_index()#.apply(lambda x: f'{x:.2f}').reset_index()
avgprofit2.columns = ['ticker', 'strategy', 'timeframe', 'AvgProfitPerTrade%']
prof12 = avgprofit2.merge(totalprofit2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])

#Computing the lowest value of the cumulative profit (after each trade)
mincumprofitp2 = data.groupby(['ticker', 'strategy', 'timeframe'])['CumulativeProfitStrategy%'].min().reset_index()
mincumprofitp2.columns = ['ticker', 'strategy', 'timeframe', 'MinCumulative%']
prof22 = mincumprofitp2.merge(prof12, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])

#Computing the maximum value of the cumulative profit (after each trade)
maxcumprofitp2 = data.groupby(['ticker', 'strategy', 'timeframe'])['CumulativeProfitStrategy%'].max().reset_index()
maxcumprofitp2.columns = ['ticker', 'strategy', 'timeframe', 'MaxCumulative%']
profit2 = maxcumprofitp2.merge(prof22, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
profit2 #check that it works

,ticker,strategy,timeframe,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,-0.28%,-0.00%,-0.000727,-0.283710
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,-0.28%,-0.00%,-0.000634,-0.277620
2,BINANCE:1000PEPEUSDT.P,Strategy v2.1,1s,-0.24%,-0.00%,-0.000796,-0.243668
3,BINANCE:1000SHIBUSDT.P,Strategy v2.1,1s,-0.27%,-0.00%,-0.000790,-0.271641
4,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,0.23%,-0.00%,0.000449,0.188500
...,...,...,...,...,...,...,...
527,BINANCE:YFIUSDT.P,Strategy v2.1,1s,999.94%,1009.93%,15.629635,7189.632000
528,BINANCE:ZECUSDT.P,Strategy v2.1,1s,9.88%,-0.03%,0.057782,22.824000
529,BINANCE:ZENUSDT.P,Strategy v2.1,1s,9.99%,-0.08%,0.028252,13.108800
530,BINANCE:ZILUSDT.P,Strategy v2.1,1s,-0.25%,-0.00%,-0.000755,-0.251450


In [6]:
################################################################################################################################
#Determine the interval in which the Strategy has been active for the ticker and timeframe and get the result in days
MinDate2 = data.groupby(['ticker', 'strategy', 'timeframe'])['entry_time'].min().reset_index()
MaxDate2 = data.groupby(['ticker', 'strategy', 'timeframe'])['exit_time'].max().reset_index()
m2 = MaxDate2.merge(profit2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2 = MinDate2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2['TimeInDays'] = (m2.exit_time - m2.entry_time).dt.total_seconds() / 86400
m2

,ticker,strategy,timeframe,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,2023-07-05 00:22:00,2023-07-26 00:34:02,-0.28%,-0.00%,-0.000727,-0.283710,21.008356
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,2023-07-06 00:11:00,2023-07-26 03:10:17,-0.28%,-0.00%,-0.000634,-0.277620,20.124502
2,BINANCE:1000PEPEUSDT.P,Strategy v2.1,1s,2023-07-06 00:07:00,2023-07-26 00:35:59,-0.24%,-0.00%,-0.000796,-0.243668,20.020127
3,BINANCE:1000SHIBUSDT.P,Strategy v2.1,1s,2023-07-06 00:15:00,2023-07-26 08:20:26,-0.27%,-0.00%,-0.000790,-0.271641,20.337106
4,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,2023-07-05 00:17:00,2023-07-26 08:53:58,0.23%,-0.00%,0.000449,0.188500,21.359005
...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:YFIUSDT.P,Strategy v2.1,1s,2023-07-06 00:14:00,2023-07-26 05:55:58,999.94%,1009.93%,15.629635,7189.632000,20.237477
528,BINANCE:ZECUSDT.P,Strategy v2.1,1s,2023-07-06 00:34:00,2023-07-26 03:00:35,9.88%,-0.03%,0.057782,22.824000,20.101794
529,BINANCE:ZENUSDT.P,Strategy v2.1,1s,2023-07-06 00:11:00,2023-07-26 03:17:51,9.99%,-0.08%,0.028252,13.108800,20.129757
530,BINANCE:ZILUSDT.P,Strategy v2.1,1s,2023-07-06 00:18:00,2023-07-26 01:06:33,-0.25%,-0.00%,-0.000755,-0.251450,20.033715


In [7]:
################################################################################################################################
#Compute the number of trades the strategy has placed for the ticker and timeframe within the interval
Trades2 = pd.DataFrame()
Trades2 = data.groupby(['ticker', 'strategy', 'timeframe']).size().reset_index()
Trades2.columns = ['ticker', 'strategy', 'timeframe', 'NoOfTrades']
m2 = Trades2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,NoOfTrades,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,390,2023-07-05 00:22:00,2023-07-26 00:34:02,-0.28%,-0.00%,-0.000727,-0.283710,21.008356
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,438,2023-07-06 00:11:00,2023-07-26 03:10:17,-0.28%,-0.00%,-0.000634,-0.277620,20.124502
2,BINANCE:1000PEPEUSDT.P,Strategy v2.1,1s,306,2023-07-06 00:07:00,2023-07-26 00:35:59,-0.24%,-0.00%,-0.000796,-0.243668,20.020127
3,BINANCE:1000SHIBUSDT.P,Strategy v2.1,1s,344,2023-07-06 00:15:00,2023-07-26 08:20:26,-0.27%,-0.00%,-0.000790,-0.271641,20.337106
4,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,420,2023-07-05 00:17:00,2023-07-26 08:53:58,0.23%,-0.00%,0.000449,0.188500,21.359005
...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:YFIUSDT.P,Strategy v2.1,1s,460,2023-07-06 00:14:00,2023-07-26 05:55:58,999.94%,1009.93%,15.629635,7189.632000,20.237477
528,BINANCE:ZECUSDT.P,Strategy v2.1,1s,395,2023-07-06 00:34:00,2023-07-26 03:00:35,9.88%,-0.03%,0.057782,22.824000,20.101794
529,BINANCE:ZENUSDT.P,Strategy v2.1,1s,464,2023-07-06 00:11:00,2023-07-26 03:17:51,9.99%,-0.08%,0.028252,13.108800,20.129757
530,BINANCE:ZILUSDT.P,Strategy v2.1,1s,333,2023-07-06 00:18:00,2023-07-26 01:06:33,-0.25%,-0.00%,-0.000755,-0.251450,20.033715


In [8]:
################################################################################################################################
#Get the average drawdown of the strategy for the tuple/timeframe
drawdown2  = pd.DataFrame()
drawdown2 = data.groupby(['ticker', 'strategy', 'timeframe'])['drawdown%'].mean().apply(lambda x: f'{x:.1f}').reset_index()
m2 = drawdown2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,drawdown%,NoOfTrades,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,1.1,390,2023-07-05 00:22:00,2023-07-26 00:34:02,-0.28%,-0.00%,-0.000727,-0.283710,21.008356
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,1.1,438,2023-07-06 00:11:00,2023-07-26 03:10:17,-0.28%,-0.00%,-0.000634,-0.277620,20.124502
2,BINANCE:1000PEPEUSDT.P,Strategy v2.1,1s,1.3,306,2023-07-06 00:07:00,2023-07-26 00:35:59,-0.24%,-0.00%,-0.000796,-0.243668,20.020127
3,BINANCE:1000SHIBUSDT.P,Strategy v2.1,1s,1.0,344,2023-07-06 00:15:00,2023-07-26 08:20:26,-0.27%,-0.00%,-0.000790,-0.271641,20.337106
4,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,1.0,420,2023-07-05 00:17:00,2023-07-26 08:53:58,0.23%,-0.00%,0.000449,0.188500,21.359005
...,...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:YFIUSDT.P,Strategy v2.1,1s,0.9,460,2023-07-06 00:14:00,2023-07-26 05:55:58,999.94%,1009.93%,15.629635,7189.632000,20.237477
528,BINANCE:ZECUSDT.P,Strategy v2.1,1s,1.0,395,2023-07-06 00:34:00,2023-07-26 03:00:35,9.88%,-0.03%,0.057782,22.824000,20.101794
529,BINANCE:ZENUSDT.P,Strategy v2.1,1s,1.3,464,2023-07-06 00:11:00,2023-07-26 03:17:51,9.99%,-0.08%,0.028252,13.108800,20.129757
530,BINANCE:ZILUSDT.P,Strategy v2.1,1s,1.1,333,2023-07-06 00:18:00,2023-07-26 01:06:33,-0.25%,-0.00%,-0.000755,-0.251450,20.033715


In [9]:
################################################################################################################################
#Get the average runup of the strategy for the tuple/timeframe
runup2 = pd.DataFrame()
runup2 = data.groupby(['ticker', 'strategy', 'timeframe'])['runup%'].mean().apply(lambda x: f'{x:.1f}').reset_index()
runup2.columns = ['ticker', 'strategy', 'timeframe', 'runup%']
m2 = runup2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,runup%,drawdown%,NoOfTrades,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,0.9,1.1,390,2023-07-05 00:22:00,2023-07-26 00:34:02,-0.28%,-0.00%,-0.000727,-0.283710,21.008356
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,0.6,1.1,438,2023-07-06 00:11:00,2023-07-26 03:10:17,-0.28%,-0.00%,-0.000634,-0.277620,20.124502
2,BINANCE:1000PEPEUSDT.P,Strategy v2.1,1s,0.6,1.3,306,2023-07-06 00:07:00,2023-07-26 00:35:59,-0.24%,-0.00%,-0.000796,-0.243668,20.020127
3,BINANCE:1000SHIBUSDT.P,Strategy v2.1,1s,0.5,1.0,344,2023-07-06 00:15:00,2023-07-26 08:20:26,-0.27%,-0.00%,-0.000790,-0.271641,20.337106
4,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,0.7,1.0,420,2023-07-05 00:17:00,2023-07-26 08:53:58,0.23%,-0.00%,0.000449,0.188500,21.359005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:YFIUSDT.P,Strategy v2.1,1s,0.6,0.9,460,2023-07-06 00:14:00,2023-07-26 05:55:58,999.94%,1009.93%,15.629635,7189.632000,20.237477
528,BINANCE:ZECUSDT.P,Strategy v2.1,1s,0.6,1.0,395,2023-07-06 00:34:00,2023-07-26 03:00:35,9.88%,-0.03%,0.057782,22.824000,20.101794
529,BINANCE:ZENUSDT.P,Strategy v2.1,1s,0.8,1.3,464,2023-07-06 00:11:00,2023-07-26 03:17:51,9.99%,-0.08%,0.028252,13.108800,20.129757
530,BINANCE:ZILUSDT.P,Strategy v2.1,1s,0.7,1.1,333,2023-07-06 00:18:00,2023-07-26 01:06:33,-0.25%,-0.00%,-0.000755,-0.251450,20.033715


In [10]:
################################################################################################################################
#Get the average time in market (in hours) of the strategy for the ticker/timeframe
avgtimeinmkt2 = pd.DataFrame()
avgtimeinmkt2 = data.groupby(['ticker', 'strategy', 'timeframe'])['TimeInMarket'].mean().reset_index()
avgtimeinmkt2.columns = ['ticker', 'strategy', 'timeframe', 'AvgTimeInMarketHours']
avgtimeinmkt2['AvgTimeInMarketHours'] = (avgtimeinmkt2['AvgTimeInMarketHours'] / 60)
m2 = avgtimeinmkt2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,AvgTimeInMarketHours,runup%,drawdown%,NoOfTrades,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,0.081137,0.9,1.1,390,2023-07-05 00:22:00,2023-07-26 00:34:02,-0.28%,-0.00%,-0.000727,-0.283710,21.008356
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,0.061670,0.6,1.1,438,2023-07-06 00:11:00,2023-07-26 03:10:17,-0.28%,-0.00%,-0.000634,-0.277620,20.124502
2,BINANCE:1000PEPEUSDT.P,Strategy v2.1,1s,0.048764,0.6,1.3,306,2023-07-06 00:07:00,2023-07-26 00:35:59,-0.24%,-0.00%,-0.000796,-0.243668,20.020127
3,BINANCE:1000SHIBUSDT.P,Strategy v2.1,1s,0.067100,0.5,1.0,344,2023-07-06 00:15:00,2023-07-26 08:20:26,-0.27%,-0.00%,-0.000790,-0.271641,20.337106
4,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,0.049969,0.7,1.0,420,2023-07-05 00:17:00,2023-07-26 08:53:58,0.23%,-0.00%,0.000449,0.188500,21.359005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:YFIUSDT.P,Strategy v2.1,1s,0.060596,0.6,0.9,460,2023-07-06 00:14:00,2023-07-26 05:55:58,999.94%,1009.93%,15.629635,7189.632000,20.237477
528,BINANCE:ZECUSDT.P,Strategy v2.1,1s,0.063033,0.6,1.0,395,2023-07-06 00:34:00,2023-07-26 03:00:35,9.88%,-0.03%,0.057782,22.824000,20.101794
529,BINANCE:ZENUSDT.P,Strategy v2.1,1s,0.061434,0.8,1.3,464,2023-07-06 00:11:00,2023-07-26 03:17:51,9.99%,-0.08%,0.028252,13.108800,20.129757
530,BINANCE:ZILUSDT.P,Strategy v2.1,1s,0.072842,0.7,1.1,333,2023-07-06 00:18:00,2023-07-26 01:06:33,-0.25%,-0.00%,-0.000755,-0.251450,20.033715


In [11]:
################################################################################################################################
#Get the standard deviation of the time in market (in hours) of the strategy for the tuple/timeframe
stdtimeinmkt2 = pd.DataFrame()
stdtimeinmkt2 = data.groupby(['ticker', 'strategy', 'timeframe'])['TimeInMarket'].std().reset_index()
stdtimeinmkt2.columns = ['ticker', 'strategy', 'timeframe', 'StdTimeInMarket']
stdtimeinmkt2['StdTimeInMarket'] = (stdtimeinmkt2['StdTimeInMarket'] / 60)
m2 = stdtimeinmkt2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,StdTimeInMarket,AvgTimeInMarketHours,runup%,drawdown%,NoOfTrades,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,0.087805,0.081137,0.9,1.1,390,2023-07-05 00:22:00,2023-07-26 00:34:02,-0.28%,-0.00%,-0.000727,-0.283710,21.008356
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,0.086208,0.061670,0.6,1.1,438,2023-07-06 00:11:00,2023-07-26 03:10:17,-0.28%,-0.00%,-0.000634,-0.277620,20.124502
2,BINANCE:1000PEPEUSDT.P,Strategy v2.1,1s,0.082377,0.048764,0.6,1.3,306,2023-07-06 00:07:00,2023-07-26 00:35:59,-0.24%,-0.00%,-0.000796,-0.243668,20.020127
3,BINANCE:1000SHIBUSDT.P,Strategy v2.1,1s,0.095664,0.067100,0.5,1.0,344,2023-07-06 00:15:00,2023-07-26 08:20:26,-0.27%,-0.00%,-0.000790,-0.271641,20.337106
4,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,0.079573,0.049969,0.7,1.0,420,2023-07-05 00:17:00,2023-07-26 08:53:58,0.23%,-0.00%,0.000449,0.188500,21.359005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:YFIUSDT.P,Strategy v2.1,1s,0.082937,0.060596,0.6,0.9,460,2023-07-06 00:14:00,2023-07-26 05:55:58,999.94%,1009.93%,15.629635,7189.632000,20.237477
528,BINANCE:ZECUSDT.P,Strategy v2.1,1s,0.097508,0.063033,0.6,1.0,395,2023-07-06 00:34:00,2023-07-26 03:00:35,9.88%,-0.03%,0.057782,22.824000,20.101794
529,BINANCE:ZENUSDT.P,Strategy v2.1,1s,0.085427,0.061434,0.8,1.3,464,2023-07-06 00:11:00,2023-07-26 03:17:51,9.99%,-0.08%,0.028252,13.108800,20.129757
530,BINANCE:ZILUSDT.P,Strategy v2.1,1s,0.100233,0.072842,0.7,1.1,333,2023-07-06 00:18:00,2023-07-26 01:06:33,-0.25%,-0.00%,-0.000755,-0.251450,20.033715


In [12]:
################################################################################################################################
#Get the min and max price within the interval (for volatility)
minentry2 = data.groupby(['ticker', 'strategy', 'timeframe'])['entry_price'].min().reset_index()
minentry2.columns = ['ticker', 'strategy', 'timeframe', 'MinPriceEntry']
m2 = minentry2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
maxentry2 = data.groupby(['ticker', 'strategy', 'timeframe'])['entry_price'].max().reset_index()
maxentry2.columns = ['ticker', 'strategy', 'timeframe', 'MaxPriceEntry']
m2 = maxentry2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
minexit2 = data.groupby(['ticker', 'strategy', 'timeframe'])['entry_price'].min().reset_index()
minexit2.columns = ['ticker', 'strategy', 'timeframe', 'MinPriceExit']
m2 = minexit2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
maxexit2 = data.groupby(['ticker', 'strategy', 'timeframe'])['entry_price'].max().reset_index()
maxexit2.columns = ['ticker', 'strategy', 'timeframe', 'MaxPriceExit']
m2 = maxexit2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2['MinPrice'] = np.where(m2['MinPriceEntry'] <= m2['MinPriceExit'], m2['MinPriceEntry'], m2['MinPriceExit'])
m2['MaxPrice'] = np.where(m2['MaxPriceEntry'] >= m2['MaxPriceExit'], m2['MaxPriceEntry'], m2['MaxPriceExit'])
m2['var'] = np.where(m2['MinPrice'] <= m2['MaxPrice'], ((m2['MaxPrice'] - m2['MinPrice']) / m2['MinPrice']), 1000)
m2['var'] = m2['var'].apply(lambda x: f'{x:.1f}%')
m2

,ticker,strategy,timeframe,MaxPriceExit,MinPriceExit,MaxPriceEntry,MinPriceEntry,StdTimeInMarket,AvgTimeInMarketHours,runup%,...,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,0.026920,0.021810,0.026920,0.021810,0.087805,0.081137,0.9,...,2023-07-05 00:22:00,2023-07-26 00:34:02,-0.28%,-0.00%,-0.000727,-0.283710,21.008356,0.021810,0.026920,0.2%
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,0.092630,0.081060,0.092630,0.081060,0.086208,0.061670,0.6,...,2023-07-06 00:11:00,2023-07-26 03:10:17,-0.28%,-0.00%,-0.000634,-0.277620,20.124502,0.081060,0.092630,0.1%
2,BINANCE:1000PEPEUSDT.P,Strategy v2.1,1s,0.001734,0.001308,0.001734,0.001308,0.082377,0.048764,0.6,...,2023-07-06 00:07:00,2023-07-26 00:35:59,-0.24%,-0.00%,-0.000796,-0.243668,20.020127,0.001308,0.001734,0.3%
3,BINANCE:1000SHIBUSDT.P,Strategy v2.1,1s,0.008380,0.007125,0.008380,0.007125,0.095664,0.067100,0.5,...,2023-07-06 00:15:00,2023-07-26 08:20:26,-0.27%,-0.00%,-0.000790,-0.271641,20.337106,0.007125,0.008380,0.2%
4,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,0.444100,0.295900,0.444100,0.295900,0.079573,0.049969,0.7,...,2023-07-05 00:17:00,2023-07-26 08:53:58,0.23%,-0.00%,0.000449,0.188500,21.359005,0.295900,0.444100,0.5%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:YFIUSDT.P,Strategy v2.1,1s,7415.000000,6464.000000,7415.000000,6464.000000,0.082937,0.060596,0.6,...,2023-07-06 00:14:00,2023-07-26 05:55:58,999.94%,1009.93%,15.629635,7189.632000,20.237477,6464.000000,7415.000000,0.1%
528,BINANCE:ZECUSDT.P,Strategy v2.1,1s,33.360000,28.150000,33.360000,28.150000,0.097508,0.063033,0.6,...,2023-07-06 00:34:00,2023-07-26 03:00:35,9.88%,-0.03%,0.057782,22.824000,20.101794,28.150000,33.360000,0.2%
529,BINANCE:ZENUSDT.P,Strategy v2.1,1s,9.266000,7.685000,9.266000,7.685000,0.085427,0.061434,0.8,...,2023-07-06 00:11:00,2023-07-26 03:17:51,9.99%,-0.08%,0.028252,13.108800,20.129757,7.685000,9.266000,0.2%
530,BINANCE:ZILUSDT.P,Strategy v2.1,1s,0.022930,0.019670,0.022930,0.019670,0.100233,0.072842,0.7,...,2023-07-06 00:18:00,2023-07-26 01:06:33,-0.25%,-0.00%,-0.000755,-0.251450,20.033715,0.019670,0.022930,0.2%


In [13]:
#Compute the number of trades with positive and negative outcome 
################################################################################################################################

negativeonly2 = pd.DataFrame()
positiveonly2 = pd.DataFrame()
negativeonly2 = data[data['profit%'] <= 0]
positiveonly2 = data[data['profit%'] > 0]

In [14]:
################################################################################################################################
#Compute percentage of negative trades for benchmarking
NoNegative2 = pd.DataFrame()
NoNegative2 = negativeonly2.groupby(['ticker', 'strategy', 'timeframe']).size().reset_index()
NoNegative2.columns = ['ticker', 'strategy', 'timeframe', 'NoOfNegative']
ProfitNegative2 = negativeonly2.groupby(['ticker', 'strategy', 'timeframe'])['profit%'].sum().reset_index()
ProfitNegative2.columns = ['ticker', 'strategy', 'timeframe', 'ProfitNegative%']
OnNegative2 = NoNegative2.merge(ProfitNegative2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2 = OnNegative2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2

,ticker,strategy,timeframe,NoOfNegative,ProfitNegative%,MaxPriceExit,MinPriceExit,MaxPriceEntry,MinPriceEntry,StdTimeInMarket,...,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,385.0,-0.288390,0.026920,0.021810,0.026920,0.021810,0.087805,...,2023-07-05 00:22:00,2023-07-26 00:34:02,-0.28%,-0.00%,-0.000727,-0.283710,21.008356,0.021810,0.026920,0.2%
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,404.0,-0.298810,0.092630,0.081060,0.092630,0.081060,0.086208,...,2023-07-06 00:11:00,2023-07-26 03:10:17,-0.28%,-0.00%,-0.000634,-0.277620,20.124502,0.081060,0.092630,0.1%
2,BINANCE:1000PEPEUSDT.P,Strategy v2.1,1s,306.0,-0.243668,0.001734,0.001308,0.001734,0.001308,0.082377,...,2023-07-06 00:07:00,2023-07-26 00:35:59,-0.24%,-0.00%,-0.000796,-0.243668,20.020127,0.001308,0.001734,0.3%
3,BINANCE:1000SHIBUSDT.P,Strategy v2.1,1s,344.0,-0.271641,0.008380,0.007125,0.008380,0.007125,0.095664,...,2023-07-06 00:15:00,2023-07-26 08:20:26,-0.27%,-0.00%,-0.000790,-0.271641,20.337106,0.007125,0.008380,0.2%
4,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,139.0,-0.423800,0.444100,0.295900,0.444100,0.295900,0.079573,...,2023-07-05 00:17:00,2023-07-26 08:53:58,0.23%,-0.00%,0.000449,0.188500,21.359005,0.295900,0.444100,0.5%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:STGUSDT.P,Strategy v10,1m,NaN,NaN,0.613100,0.594900,0.613100,0.594900,0.116083,...,2023-07-22 10:00:00,2023-07-31 00:18:00,0.03%,0.02%,0.015450,0.030900,8.595833,0.594900,0.613100,0.0%
528,BINANCE:XLMUSDT.P,Strategy Mix v2.1,1m,NaN,NaN,0.127970,0.127970,0.127970,0.127970,NaN,...,2023-07-18 21:39:00,2023-07-18 23:33:00,0.00%,0.00%,0.001780,0.001780,0.079167,0.127970,0.127970,0.0%
529,BINANCE:XRPUSDT.P,Strategy Mix v1.2,1m,NaN,NaN,0.704300,0.704300,0.704300,0.704300,NaN,...,2023-07-30 19:52:00,2023-07-31 19:55:00,0.01%,0.01%,0.013000,0.013000,1.002083,0.704300,0.704300,0.0%
530,BINANCE:XRPUSDT.P,Strategy Mix v2.1,1m,NaN,NaN,0.699400,0.699400,0.699400,0.699400,NaN,...,2023-07-26 07:20:00,2023-07-26 18:06:00,0.01%,0.01%,0.009300,0.009300,0.448611,0.699400,0.699400,0.0%


In [15]:
################################################################################################################################
#Compute percentage of positive trades for benchmarking
NoPositive2 = pd.DataFrame()
NoPositive2 = positiveonly2.groupby(['ticker', 'strategy', 'timeframe']).size().reset_index()
NoPositive2.columns = ['ticker', 'strategy', 'timeframe', 'NoOfPositive']
ProfitPositive2 = positiveonly2.groupby(['ticker', 'strategy', 'timeframe'])['profit%'].sum().reset_index()
ProfitPositive2.columns = ['ticker', 'strategy', 'timeframe', 'ProfitPositive%']
OnPositive2 = NoPositive2.merge(ProfitPositive2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2 = OnPositive2.merge(m2, how = 'outer', on = ['ticker', 'strategy', 'timeframe'])
m2['NoOfPositive'] = m2['NoOfPositive'].fillna(0)
m2['ProfitPositive%'] = m2['ProfitPositive%'].fillna(0)
m2['NoOfNegative'] = m2['NoOfNegative'].fillna(0)
m2['ProfitNegative%'] = m2['ProfitNegative%'].fillna(0)
m2

,ticker,strategy,timeframe,NoOfPositive,ProfitPositive%,NoOfNegative,ProfitNegative%,MaxPriceExit,MinPriceExit,MaxPriceEntry,...,entry_time,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,5.0,0.00468,385.0,-0.28839,0.02692,0.02181,0.02692,...,2023-07-05 00:22:00,2023-07-26 00:34:02,-0.28%,-0.00%,-0.000727,-0.28371,21.008356,0.02181,0.02692,0.2%
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,34.0,0.02119,404.0,-0.29881,0.09263,0.08106,0.09263,...,2023-07-06 00:11:00,2023-07-26 03:10:17,-0.28%,-0.00%,-0.000634,-0.27762,20.124502,0.08106,0.09263,0.1%
2,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,281.0,0.61230,139.0,-0.42380,0.44410,0.29590,0.44410,...,2023-07-05 00:17:00,2023-07-26 08:53:58,0.23%,-0.00%,0.000449,0.18850,21.359005,0.29590,0.44410,0.5%
3,BINANCE:AAVEUSDT.P,Strategy v2.1,1s,333.0,155.06360,77.0,-73.22160,80.48000,69.24000,80.48000,...,2023-07-06 00:07:00,2023-07-26 02:59:03,9.89%,-0.14%,0.199615,81.84200,20.119479,69.24000,80.48000,0.2%
4,BINANCE:ADAUSDT.P,Strategy v1,1m,2.0,0.04470,1.0,-0.00160,0.35100,0.30490,0.35100,...,2023-07-13 15:48:00,2023-07-24 10:02:00,0.04%,-0.00%,0.014367,0.04310,10.759722,0.30490,0.35100,0.2%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:TRXUSDT.P,Strategy v2.1,1s,0.0,0.00000,307.0,-0.23781,0.08444,0.07674,0.08444,...,2023-07-06 00:12:00,2023-07-26 09:37:16,-0.24%,-0.00%,-0.000775,-0.23781,20.392546,0.07674,0.08444,0.1%
528,BINANCE:TUSDT.P,Strategy v2.1,1s,0.0,0.00000,299.0,-0.22530,0.02492,0.02200,0.02492,...,2023-07-05 00:11:00,2023-07-25 00:41:41,-0.23%,-0.00%,-0.000754,-0.22530,20.021308,0.02200,0.02492,0.1%
529,BINANCE:VETUSDT.P,Strategy v2.1,1s,0.0,0.00000,332.0,-0.26044,0.02030,0.01805,0.02030,...,2023-07-05 00:20:00,2023-07-26 13:11:59,-0.26%,-0.00%,-0.000784,-0.26044,21.536100,0.01805,0.02030,0.1%
530,BINANCE:XRPUSDT.P,Strategy Mix v1.1,1m,0.0,0.00000,1.0,-0.03800,0.77010,0.77010,0.77010,...,2023-07-14 07:35:00,2023-07-15 03:23:00,-0.04%,-0.04%,-0.038000,-0.03800,0.825000,0.77010,0.77010,0.0%


In [16]:
################################################################################################################################
#Compute ratio [-100 to 100] as division of profit made from positive trades divided by profit made from negative trades. If not negative trades, the ratio is 100
#Similar to Profit Factor in TradingView Statistics
m2['ratio'] = np.where(m2['ProfitNegative%'] != 0.00, m2['ProfitPositive%'] / (m2['ProfitNegative%'] * -1), 100)
m2['ratio'] = np.where(m2['ProfitPositive%'] == 0, -100, m2['ratio'])
m2['ratio'] = m2['ratio'].apply(lambda x: f'{x:.1f}')
m2

,ticker,strategy,timeframe,NoOfPositive,ProfitPositive%,NoOfNegative,ProfitNegative%,MaxPriceExit,MinPriceExit,MaxPriceEntry,...,exit_time,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var,ratio
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,5.0,0.00468,385.0,-0.28839,0.02692,0.02181,0.02692,...,2023-07-26 00:34:02,-0.28%,-0.00%,-0.000727,-0.28371,21.008356,0.02181,0.02692,0.2%,0.0
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,34.0,0.02119,404.0,-0.29881,0.09263,0.08106,0.09263,...,2023-07-26 03:10:17,-0.28%,-0.00%,-0.000634,-0.27762,20.124502,0.08106,0.09263,0.1%,0.1
2,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,281.0,0.61230,139.0,-0.42380,0.44410,0.29590,0.44410,...,2023-07-26 08:53:58,0.23%,-0.00%,0.000449,0.18850,21.359005,0.29590,0.44410,0.5%,1.4
3,BINANCE:AAVEUSDT.P,Strategy v2.1,1s,333.0,155.06360,77.0,-73.22160,80.48000,69.24000,80.48000,...,2023-07-26 02:59:03,9.89%,-0.14%,0.199615,81.84200,20.119479,69.24000,80.48000,0.2%,2.1
4,BINANCE:ADAUSDT.P,Strategy v1,1m,2.0,0.04470,1.0,-0.00160,0.35100,0.30490,0.35100,...,2023-07-24 10:02:00,0.04%,-0.00%,0.014367,0.04310,10.759722,0.30490,0.35100,0.2%,27.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:TRXUSDT.P,Strategy v2.1,1s,0.0,0.00000,307.0,-0.23781,0.08444,0.07674,0.08444,...,2023-07-26 09:37:16,-0.24%,-0.00%,-0.000775,-0.23781,20.392546,0.07674,0.08444,0.1%,-100.0
528,BINANCE:TUSDT.P,Strategy v2.1,1s,0.0,0.00000,299.0,-0.22530,0.02492,0.02200,0.02492,...,2023-07-25 00:41:41,-0.23%,-0.00%,-0.000754,-0.22530,20.021308,0.02200,0.02492,0.1%,-100.0
529,BINANCE:VETUSDT.P,Strategy v2.1,1s,0.0,0.00000,332.0,-0.26044,0.02030,0.01805,0.02030,...,2023-07-26 13:11:59,-0.26%,-0.00%,-0.000784,-0.26044,21.536100,0.01805,0.02030,0.1%,-100.0
530,BINANCE:XRPUSDT.P,Strategy Mix v1.1,1m,0.0,0.00000,1.0,-0.03800,0.77010,0.77010,0.77010,...,2023-07-15 03:23:00,-0.04%,-0.04%,-0.038000,-0.03800,0.825000,0.77010,0.77010,0.0%,-100.0


In [17]:
################################################################################################################################
#Ratio of negative trades out of total number of trades
m2['RatioNegative'] = np.where(m2['NoOfTrades'] > 0, m2['NoOfNegative'] / m2['NoOfTrades'], 0)
m2['RatioNegative'] = m2['RatioNegative'].apply(lambda x: f'{x:.0%}')
m2

,ticker,strategy,timeframe,NoOfPositive,ProfitPositive%,NoOfNegative,ProfitNegative%,MaxPriceExit,MinPriceExit,MaxPriceEntry,...,MaxCumulative%,MinCumulative%,AvgProfitPerTrade%,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var,ratio,RatioNegative
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,5.0,0.00468,385.0,-0.28839,0.02692,0.02181,0.02692,...,-0.28%,-0.00%,-0.000727,-0.28371,21.008356,0.02181,0.02692,0.2%,0.0,99%
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,34.0,0.02119,404.0,-0.29881,0.09263,0.08106,0.09263,...,-0.28%,-0.00%,-0.000634,-0.27762,20.124502,0.08106,0.09263,0.1%,0.1,92%
2,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,281.0,0.61230,139.0,-0.42380,0.44410,0.29590,0.44410,...,0.23%,-0.00%,0.000449,0.18850,21.359005,0.29590,0.44410,0.5%,1.4,33%
3,BINANCE:AAVEUSDT.P,Strategy v2.1,1s,333.0,155.06360,77.0,-73.22160,80.48000,69.24000,80.48000,...,9.89%,-0.14%,0.199615,81.84200,20.119479,69.24000,80.48000,0.2%,2.1,19%
4,BINANCE:ADAUSDT.P,Strategy v1,1m,2.0,0.04470,1.0,-0.00160,0.35100,0.30490,0.35100,...,0.04%,-0.00%,0.014367,0.04310,10.759722,0.30490,0.35100,0.2%,27.9,33%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:TRXUSDT.P,Strategy v2.1,1s,0.0,0.00000,307.0,-0.23781,0.08444,0.07674,0.08444,...,-0.24%,-0.00%,-0.000775,-0.23781,20.392546,0.07674,0.08444,0.1%,-100.0,100%
528,BINANCE:TUSDT.P,Strategy v2.1,1s,0.0,0.00000,299.0,-0.22530,0.02492,0.02200,0.02492,...,-0.23%,-0.00%,-0.000754,-0.22530,20.021308,0.02200,0.02492,0.1%,-100.0,100%
529,BINANCE:VETUSDT.P,Strategy v2.1,1s,0.0,0.00000,332.0,-0.26044,0.02030,0.01805,0.02030,...,-0.26%,-0.00%,-0.000784,-0.26044,21.536100,0.01805,0.02030,0.1%,-100.0,100%
530,BINANCE:XRPUSDT.P,Strategy Mix v1.1,1m,0.0,0.00000,1.0,-0.03800,0.77010,0.77010,0.77010,...,-0.04%,-0.04%,-0.038000,-0.03800,0.825000,0.77010,0.77010,0.0%,-100.0,100%


In [18]:
################################################################################################################################
#Adjustment in case of using leverage: settings are x100 for FX and x10 for crypto
m2['AdjustedProfit%'] = np.where(m2['ticker'].str.contains('FX:'), m2['TotalProfit%'] * 100, m2['TotalProfit%'])
m2['AdjustedProfit%'] = np.where(m2['ticker'].str.contains('BINANCE:') | m2['ticker'].str.contains('BITMEX:'), m2['TotalProfit%'] * 10, m2['AdjustedProfit%'])

#Compute the average adjusted profit% per day
m2['APPD'] = m2.apply(lambda row: row['AdjustedProfit%'] / row.TimeInDays, axis = 1)
m2['TradesPerDay'] = np.where(m2['TimeInDays'] >= 1, m2['NoOfTrades'] / m2['TimeInDays'], 0)
m2['TradesPerDay'] = m2['TradesPerDay'].apply(lambda x: f'{x:.1f}')
m2

,ticker,strategy,timeframe,NoOfPositive,ProfitPositive%,NoOfNegative,ProfitNegative%,MaxPriceExit,MinPriceExit,MaxPriceEntry,...,TotalProfit%,TimeInDays,MinPrice,MaxPrice,var,ratio,RatioNegative,AdjustedProfit%,APPD,TradesPerDay
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,5.0,0.00468,385.0,-0.28839,0.02692,0.02181,0.02692,...,-0.28371,21.008356,0.02181,0.02692,0.2%,0.0,99%,-2.8371,-0.135046,18.6
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,34.0,0.02119,404.0,-0.29881,0.09263,0.08106,0.09263,...,-0.27762,20.124502,0.08106,0.09263,0.1%,0.1,92%,-2.7762,-0.137951,21.8
2,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,281.0,0.61230,139.0,-0.42380,0.44410,0.29590,0.44410,...,0.18850,21.359005,0.29590,0.44410,0.5%,1.4,33%,1.8850,0.088253,19.7
3,BINANCE:AAVEUSDT.P,Strategy v2.1,1s,333.0,155.06360,77.0,-73.22160,80.48000,69.24000,80.48000,...,81.84200,20.119479,69.24000,80.48000,0.2%,2.1,19%,818.4200,40.677991,20.4
4,BINANCE:ADAUSDT.P,Strategy v1,1m,2.0,0.04470,1.0,-0.00160,0.35100,0.30490,0.35100,...,0.04310,10.759722,0.30490,0.35100,0.2%,27.9,33%,0.4310,0.040057,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:TRXUSDT.P,Strategy v2.1,1s,0.0,0.00000,307.0,-0.23781,0.08444,0.07674,0.08444,...,-0.23781,20.392546,0.07674,0.08444,0.1%,-100.0,100%,-2.3781,-0.116616,15.1
528,BINANCE:TUSDT.P,Strategy v2.1,1s,0.0,0.00000,299.0,-0.22530,0.02492,0.02200,0.02492,...,-0.22530,20.021308,0.02200,0.02492,0.1%,-100.0,100%,-2.2530,-0.112530,14.9
529,BINANCE:VETUSDT.P,Strategy v2.1,1s,0.0,0.00000,332.0,-0.26044,0.02030,0.01805,0.02030,...,-0.26044,21.536100,0.01805,0.02030,0.1%,-100.0,100%,-2.6044,-0.120932,15.4
530,BINANCE:XRPUSDT.P,Strategy Mix v1.1,1m,0.0,0.00000,1.0,-0.03800,0.77010,0.77010,0.77010,...,-0.03800,0.825000,0.77010,0.77010,0.0%,-100.0,100%,-0.3800,-0.460606,0.0


In [19]:
################################################################################################################################
#Reorder the columns
cols2 = ['ticker', 'strategy', 'timeframe', 'entry_time', 'exit_time', 'TimeInDays', 
        'AvgTimeInMarketHours', 'StdTimeInMarket', 'NoOfTrades', 'RatioNegative', 'TotalProfit%', 'AvgProfitPerTrade%', 
        'ProfitPositive%', 'ProfitNegative%', 'ratio', 'MinCumulative%', 'MaxCumulative%', 'runup%', 'drawdown%', 
        'MinPrice', 'MaxPrice', 'var', 'TradesPerDay', 'AdjustedProfit%', 'APPD']

################################################################################################################################
#slice the df
df2 = m2[cols2]

In [20]:
################################################################################################################################
#sort by ratio
df2.sort_values(by=['ratio'], ascending=[False])

################################################################################################################################
#Save the analysis daily
os.chdir(r'D:\TD\FMF\Strategies\Results')
df2.to_csv('AllResults_OverallStrategy ' + today + '.csv', sep=',', index=False, header=True)

In [21]:
df2 #have a final look before starting to look at it in Excel

,ticker,strategy,timeframe,entry_time,exit_time,TimeInDays,AvgTimeInMarketHours,StdTimeInMarket,NoOfTrades,RatioNegative,...,MinCumulative%,MaxCumulative%,runup%,drawdown%,MinPrice,MaxPrice,var,TradesPerDay,AdjustedProfit%,APPD
0,BINANCE:1000FLOKIUSDT.P,Strategy v2.1,1s,2023-07-05 00:22:00,2023-07-26 00:34:02,21.008356,0.081137,0.087805,390,99%,...,-0.00%,-0.28%,0.9,1.1,0.02181,0.02692,0.2%,18.6,-2.8371,-0.135046
1,BINANCE:1000LUNCUSDT.P,Strategy v2.1,1s,2023-07-06 00:11:00,2023-07-26 03:10:17,20.124502,0.061670,0.086208,438,92%,...,-0.00%,-0.28%,0.6,1.1,0.08106,0.09263,0.1%,21.8,-2.7762,-0.137951
2,BINANCE:1INCHUSDT.P,Strategy v2.1,1s,2023-07-05 00:17:00,2023-07-26 08:53:58,21.359005,0.049969,0.079573,420,33%,...,-0.00%,0.23%,0.7,1.0,0.29590,0.44410,0.5%,19.7,1.8850,0.088253
3,BINANCE:AAVEUSDT.P,Strategy v2.1,1s,2023-07-06 00:07:00,2023-07-26 02:59:03,20.119479,0.058998,0.087377,410,19%,...,-0.14%,9.89%,0.7,1.0,69.24000,80.48000,0.2%,20.4,818.4200,40.677991
4,BINANCE:ADAUSDT.P,Strategy v1,1m,2023-07-13 15:48:00,2023-07-24 10:02:00,10.759722,4.288333,0.025278,3,33%,...,-0.00%,0.04%,6.6,18.8,0.30490,0.35100,0.2%,0.3,0.4310,0.040057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,BINANCE:TRXUSDT.P,Strategy v2.1,1s,2023-07-06 00:12:00,2023-07-26 09:37:16,20.392546,0.068779,0.088754,307,100%,...,-0.00%,-0.24%,0.3,0.7,0.07674,0.08444,0.1%,15.1,-2.3781,-0.116616
528,BINANCE:TUSDT.P,Strategy v2.1,1s,2023-07-05 00:11:00,2023-07-25 00:41:41,20.021308,0.091276,0.098138,299,100%,...,-0.00%,-0.23%,0.8,1.6,0.02200,0.02492,0.1%,14.9,-2.2530,-0.112530
529,BINANCE:VETUSDT.P,Strategy v2.1,1s,2023-07-05 00:20:00,2023-07-26 13:11:59,21.536100,0.082313,0.097450,332,100%,...,-0.00%,-0.26%,0.5,1.1,0.01805,0.02030,0.1%,15.4,-2.6044,-0.120932
530,BINANCE:XRPUSDT.P,Strategy Mix v1.1,1m,2023-07-14 07:35:00,2023-07-15 03:23:00,0.825000,0.330000,NaN,1,100%,...,-0.04%,-0.04%,5.1,13.4,0.77010,0.77010,0.0%,0.0,-0.3800,-0.460606
